# Jupiter Trojans and Hildas Animation

Plot the positions of planets out to Jupiter and the Trojan and Hilda asteroids for a full orbit, creating
still frames for a movie to be created using iMovie or `ffmpeg` into video files.

The combo I use is `ffmpeg` for initial assembly:
> `ffmpeg -framerate 15 -i frame%03d.png ../video.mp4`

Then read `video.mp4` into `iMovie` and export as a new mp4, adding intro/outro as needed.

The animation will run for one full Jupiter sidereal period of 4332.589 days, 225 frames at 15 frames/second

Uses `astroquery` to get data from the MPC and JPL Horizons databases.

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

# astropy bits we need

from astropy.time import Time

# used for Newton-Raphson root solving for this plot

from scipy.optimize import newton

# astroquery

from astroquery.mpc import MPC
from astroquery.jplhorizons import Horizons

# suppress nuisance warnings

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

#### Aspect Ratio

`aspect` sets the aspect ratio, width/height.  The default aspect ratio is 4:3, 1:1 for square plots, 
and 5:2 (2.5:1) for spectra, and equal aspect ratio for images and plots of Cartesian coordinates.
Use a formula instead of decimal when the result is a fraction (example: `aspect = 4.0/3.0` instead of
`aspect=1.33`).

In [ ]:
# Aspect ratio
#
# Exceptions:
#    spectra use full width we use 5:2=2.5 aspect ratio
#
# graphic aspect ratio = width/height

aspect = 1.0 #16.0/9.0

#
# Don't change these unless you really need to (we never have)
#
# fPage is the horizontal fraction of the page occupied by the figure, default 1.0
#
# scaleFac is the LaTeX includegraphics scaling in units of \textwidth, default 1.0
#

fPage = 1.0
scaleFac = 1.0

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# Graphic dimensions depending on bitmap or vector format (draft vs production)

figFmt = 'png'
dpi = 600
plotWidth = dpi*fPage*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 7
lwidth = 0.5
axisPad = 5
wInches = fPage*textWidth # float(plotWidth)/float(dpi)
hInches = wInches/aspect  # float(plotHeight)/float(dpi)

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad']=f'{axisPad}'
plt.rcParams['ytick.major.pad']=f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## convenience functions

a_pq(a_p,p,q) = semi-major axis of a p:q resonance for perturber at a_p


In [ ]:
def a_pq(a_p,p,q):
    return a_p*(p/q)**(2./3.)

# Plot an unfilled circle of a given radius

def plotCircle(ax,xc,yc,r,color,ltype='-',alpha=0):
    theta = np.linspace(0,2*np.pi,360,endpoint=True)
    xs = xc + r*np.cos(theta)
    ys = yc + r*np.sin(theta)
    ax.plot(xs,ys,linestyle=ltype,color=color,alpha=alpha,lw=0.5)

## Orbit calculation and transform functions

### Eccentric anomaly, $E$

The mean anomaly $M$ is related to the eccentric anomaly $E$ and eccentricity $e$ by Kepler's equation:
 > $M = E - e\sin(E)$

Kepler's equation cannot be solved analytially to compute $E$ given $M$ and $e$ from the orbit elements, so
we solve it numerically using the Newton-Raphson method.  

Define a function $f(E)$
 > $f(E) = E - e\sin(E) - M$

with derivative
 > $\frac{dfE}{dE} = 1 - e\cos(E)$

And solve for $f(E)=0$ given $M$ and $e$.  We will use the `scipy.optimize.newton()` function to solve
for the roots using the Newton-Raphson method.

### True anomaly, $\nu$

Compute the true anomaly $\nu$ given the eccentric anomaly $E$ and orbit eccentricity $e$, using a form that
is numerically safe when E is near $\pm\pi$:
\begin{equation}
  \nu = E + 2\arctan\left(\frac{\beta\sin E}{1-\beta\cos E}\right)
\end{equation}
where
\begin{equation}
  \beta = \frac{e}{1+(1-e^2)^{1/2}}
\end{equation}

### Orbital elements to ecliptic reference plane (x,y,z) coordinates

For a given epoch and osculating orbit elements (a,e,i,n,$\Omega$,$\omega$), `orbXYZ()` computes the
ecliptic reference plane (x,y,z) location of the object at that epoch. It solves Kepler's equation for the
eccentric anomaly $E$ and from that and the true anomaly $\nu$ calculates the location in perifocal (orbital)
plane coordinates (x$_{\rm orb}$,y$_{\rm orb}$).

These are then transformed into ecliptic reference plane XYZ coordinates using the `orbToEclitpic()` function
which performs the Euler matrix transform using the three angles ($\Omega$,$i$,$\omega$).

### Convenience Functions

`traceOrb()` traces a complete orbit in (XYZ) ecliptic coordinates given Keplerian osculating orbit elements
(a,e,i,$Omega$,$omega$).

`corotXY()` rotates ecliptic plane (x,y) coordiate into the the co-rotating reference frame of a planet 
with ecliptic plane coordinates (xP,yP).

In [ ]:
# Kepler's equation - assumes M and E in radians

def kepler(E,M,e):
    return (E - e*np.sin(E) - M)

# Derivative of Kepler's equation, E in radians

def kepler_deriv(E,M,e):
    return (1.0 - e*np.cos(E))

# Compute the eccentric anomaly - M must be in radians

def eccAnomaly(M,e):
    E = newton(kepler, M, kepler_deriv, args=(M,e))
    return E

# Compute the true anomaly - E must be in radians

def trueAnomaly(E,e):
    beta = e/(1.0+np.sqrt(1-e*e))
    sin_nu = beta*np.sin(E)
    cos_nu = 1 - beta*np.cos(E)
    nu = E + 2.0*np.arctan2(sin_nu,cos_nu)
    return nu

# Convert orbital plane (x,y) to ecliptic plane (x,y,z)
    
def orbToEcliptic(xorb,yorb,Omega,i,omega):

    # Euler matrix - angles must be in radians
    
    m_xx = np.cos(omega)*np.cos(Omega) - np.sin(omega)*np.cos(i)*np.sin(Omega)
    m_xy = np.cos(omega)*np.sin(Omega) + np.sin(omega)*np.cos(i)*np.cos(Omega)
    m_xz = np.sin(omega)*np.sin(i)
    
    m_yx = -np.sin(omega)*np.cos(Omega) - np.cos(omega)*np.cos(i)*np.sin(Omega)
    m_yy = -np.sin(omega)*np.sin(Omega) + np.cos(omega)*np.cos(i)*np.cos(Omega)
    m_yz =  np.cos(omega)*np.sin(i)
    
    # because zorb=0 by definition, we don't need these, but keep in code to remind us they are there
    #m_zx =  np.sin(i)*np.sin(Omega)
    #m_zy = -np.sin(i)*np.cos(Omega)
    #m_zz =  np.cos(i)
    
    # compute ecliptic XYZ
    
    xEcl = xorb*m_xx + yorb*m_yx # + zorb*m_zx
    yEcl = xorb*m_xy + yorb*m_yy # + zorb*m_zy
    zEcl = xorb*m_xz + yorb*m_yz # + zorb*m_zz
    
    return xEcl,yEcl,zEcl

# Compute ecliptic (x,y,z) coordinates at epochXY given orbit elements (a,e,n,M) at epoch elEpoch

def orbXYZ(a,e,n,M,i,Omega,omega,elEpoch,epochXY):
    dT = epochXY - elEpoch # years
    dM = np.radians(dT*n*365.25) # radians
    Mep = M + dM # mean anomaly at epochXY in radians
    
    # eccentric anomaly, E, at epochXY in radians
    
    E = eccAnomaly(Mep,e)
    
    # true anomaly, nu, at epochXY in radians
    
    nu = trueAnomaly(E,e)
    
    # heliocentric radius in au
    
    r = a*(1-e*e)/(1 + e*np.cos(nu))
    
    # orbit plane (x,y), perihelion is toward +x
    
    xorb = r*np.cos(nu)
    yorb = r*np.sin(nu)
    
    x,y,z = orbToEcliptic(xorb,yorb,Omega,i,omega)

    return x,y,z

# trace a full orbit in ecliptic cartesian coordinats

def traceOrb(a,e,i,Omega,peri):
    nu = np.linspace(0.0,2.*np.pi,501)
    r = a*(1-e*e)/(1+e*np.cos(nu))
    xorb = r*np.cos(nu)
    yorb = r*np.sin(nu)
    
    x,y,z = orbToEcliptic(xorb,yorb,Omega,i,peri)

    return x,y,z

# rotate ecliptic (x,y) coordinates into the co-rotating heliocentric frame of planet at (xP,yP)

def corotXY(x,y,xP,yP):
    dX = x - xP
    dY = y - yP

    rP = np.sqrt(xP*xP + yP*yP) # heliocentric distance
    theta = np.arctan2(yP,xP)   # rotation angle

    xcr =  dX*np.cos(theta) + dY*np.sin(theta) + rP
    ycr = -dX*np.sin(theta) + dY*np.cos(theta)

    return xcr,ycr

## Get orbit elements for the asteroids

Use `astroquery` to retrieve asteroids from the MPC database.  

Extract:
 * `semimajor_axis` - $a$
 * `eccentricity` - $e$
 * `inclination` - $i$
 * `argument_of_perihelion` - $\varpi$
 * `ascending_node` - $\Omega$
 * `mean_anomaly` - $M$
 * `epoch_jd` - epoch in JD (easier to work with)
 * `mean_daily_motion` - $n$
 * `orbit_type` - orbit type code
 
We select asteroids using the orbit_type codes:
 * 0: Unclassified (mostly Main Belters)
 * 1: Atiras
 * 2: Atens
 * 3: Apollos
 * 4: Amors
 * 5: Mars Crossers
 * 6: Hungarias
 * 7: Phocaeas
 * 8: Hildas
 * 9: Jupiter Trojans

and these constraints:
 * `semimajor_axis_max=5.0` - a < 5.5au to includes the Jupiter trojans

### View Date

`viewDate` defines the date for viewing the solar system.  Dates need to be in ISO8601 format with time, 
`CCYY-MM-DDThh:mm:ss` in UTC time system, for example: 
 > `2025-02-02T00:00:00`
 
is midnight UTC time on 2025 February 2.

### Date Tag

`dateTag` is an identifier added to the output PNG filenames, usually with the date.  Should be kept simple
with no spaces, for example:
 > `20250202`

corresponds to the date above. It is appended to the main filename like `InnerSolSys_20250202.png`.

You can define `dateTag` to be any non-date string to use as a unique identifier for your plots.


In [ ]:
# Starting epoch for the movie is 2025-01-01

viewDate0 = "2025-01-01T00:00:00" # Start of 2025 UTC

t = Time(viewDate0,format='isot',scale='tt') # astropy time object
viewEpoch0 = t.decimalyear
viewJD0 = t.jd

# Jupiter sidereal period in days

P_Jup = 4332.589 # days
numFrames = 225
timeStep = P_Jup/225 # days

print(f"Movie will run for {P_Jup:.3f} days, timeStep = {timeStep:.3f} days")

# constraints

Hmax = 18.0 # maximum absolute magnitude

### Jupiter Trojans

Uses MPC `orbit_type=9`

In [ ]:
# Jupiter trojans

result = MPC.query_objects('asteroid',
                           orbit_type=9,
                           absolute_magnitude_max=Hmax,
                           return_fields='semimajor_axis,eccentricity,inclination,argument_of_perihelion,ascending_node,epoch_jd,mean_anomaly,mean_daily_motion,orbit_type')

aTr = np.array([float(d.get('semimajor_axis',None)) for d in result])
eTr = np.array([float(d.get('eccentricity',None)) for d in result])
nTr = np.array([float(d.get('mean_daily_motion',None)) for d in result])
iTr = np.radians(np.array([float(d.get('inclination',None)) for d in result]))
periTr = np.radians(np.array([float(d.get('argument_of_perihelion',None)) for d in result]))
OmegaTr = np.radians(np.array([float(d.get('ascending_node',None)) for d in result]))
MTr = np.radians(np.array([float(d.get('mean_anomaly',None)) for d in result]))
epJD = np.array([d.get('epoch_jd',None) for d in result])
t = Time(epJD,format='jd',scale='tt')
epTr = t.decimalyear

print(f"{len(aTr)} Jupiter Trojans H<{Hmax:.1f}mag")

### Hildas (3:2 resonance)

Uses MPC `orbit_type=8`

In [ ]:
# Hildas

result = MPC.query_objects('asteroid',
                           orbit_type=8,absolute_magnitude_max=Hmax,
                           return_fields='semimajor_axis,eccentricity,inclination,argument_of_perihelion,ascending_node,epoch_jd,mean_anomaly,mean_daily_motion,orbit_type')

aH = np.array([float(d.get('semimajor_axis',None)) for d in result])
eH = np.array([float(d.get('eccentricity',None)) for d in result])
nH = np.array([float(d.get('mean_daily_motion',None)) for d in result])
iH = np.radians(np.array([float(d.get('inclination',None)) for d in result]))
periH = np.radians(np.array([float(d.get('argument_of_perihelion',None)) for d in result]))
OmegaH = np.radians(np.array([float(d.get('ascending_node',None)) for d in result]))
MH = np.radians(np.array([float(d.get('mean_anomaly',None)) for d in result]))
epJD = np.array([d.get('epoch_jd',None) for d in result])
t = Time(epJD,format='jd',scale='tt')
epH = t.decimalyear

print(f"{len(aH)} Hildas H<{Hmax:.1f}mag")

## Planet orbits

Retrieve the orbit elements for the outer planets plus Earth and Mars from the JPL Horizons database for
the same viewing date.  This is used to draw accurate orbits for the planets and the positions of the planets
relative to the TNOs on the viewing date

In [ ]:
planets = ['Mercury','Venus','Earth','Mars','Jupiter','Saturn','Uranus','Neptune'] 

# JPL Horizons database unique identifiers

uniqueID = {"Mercury":"Mercury Barycenter",
            "Venus":"Venus Barycenter",
            "Earth":"Earth-Moon Barycenter",
            "Mars":"Mars Barycenter",
            "Jupiter":"Jupiter Barycenter",
            "Saturn":"Saturn Barycenter",
            "Uranus":"Uranus Barycenter",
            "Neptune":"Neptune Barycenter",
            "Ceres":"Ceres",
            "Pluto":"9",
            "Eris":"20136199",
            "Haumea":"20136108",
            "Makemake":"Makemake",
            "Gonggong":"Gonggong",
            "Orcus":"20090482",
            "Quaoar":"20050000"
           }

# colors for bodies

colors = {"Mercury":"#e7e8ec",
          "Venus":"#f9c21a",
          "Earth":"#6b93d6",
          "Mars":"#c1440e",
          "Jupiter":"#c99039",
          "Saturn":"#d8ca9d",
          "Uranus":"#d1e7e7",
          "Neptune":"#85addb",
          "Ceres":"cyan",
          "Pluto":"#fff1d5",
          "Eris":"yellow",
          "Haumea":"orange",
          "Makemake":"red",
          "Gonggong":"green",
          "Orcus":"gold",
          "Quaoar":"#bb0000"}
          
xOrb = {}
yOrb = {}
zOrb = {}
aP = {}
eP = {}
iP = {}
MP = {}
OmP = {}
wP = {}
nP = {}

for body in ['Mercury','Venus','Earth','Mars','Jupiter']:
    obj = Horizons(id=f'{uniqueID[body]}',epochs=viewJD0)
    el = obj.elements()
    aP[body] = el['a'].value[0]
    eP[body]= el['e'].value[0]
    iP[body] = np.radians(el['incl'].value[0])
    MP[body] = np.radians(el['M'].value[0])
    OmP[body] = np.radians(el['Omega'].value[0])
    wP[body] = np.radians(el['w'].value[0])
    nP[body] = el['n'].value[0]
    
    xOrb[body],yOrb[body],zOrb[body] = traceOrb(aP[body],eP[body],iP[body],OmP[body],wP[body])


## Make the animation still frames

Setting `showKey=True` will turn on a color code key for the asteroids.

Set `numFrames` for the number of still frames to create.  At 15 fps, 225 frames is 15 seconds of video.

In [ ]:
aMax = 6.0
showKey = True
frameDir = "Frames"

numFrames = 225
frames = np.arange(0,numFrames+1)

for frame in frames:
    viewJD = viewJD0 + frame*timeStep

    t = Time(viewJD,format='jd',scale='tt')
    viewEpoch = t.decimalyear

    frameDate = t.tt.iso.split(' ')[0]
    frameFile = f"{frameDir}/frame{frame:03d}.png"

    # positions of the Jupiter trojans and Hildas at viewEpoch

    xTr,yTr,zTr = orbXYZ(aTr,eTr,nTr,MTr,iTr,OmegaTr,periTr,epTr,viewEpoch)
    xH,yH,zH = orbXYZ(aH,eH,nH,MH,iH,OmegaH,periH,epH,viewEpoch)

    plt.style.use('dark_background')

    fig,ax = plt.subplots(figsize=(wInches,hInches),dpi=dpi)
    
    ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
    ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

    # Limits

    ax.set_xlim(-aMax,aMax)
    ax.set_xticks([])
    ax.set_ylim(-aMax,aMax)
    ax.set_yticks([])
    ax.set_axis_off()

    ax.set_aspect(True)

    # Planet orbits

    for body in ['Mercury','Venus','Earth','Mars','Jupiter']:
        x,y,z = orbXYZ(aP[body],eP[body],nP[body],MP[body],iP[body],OmP[body],wP[body],viewEpoch0,viewEpoch)
        if body == 'Jupiter':
            ax.plot(x,y,'o',ms=4,mfc=colors[body],mec="white",mew=0.2,alpha=1,zorder=10,label='Jupiter')
        else:
            ax.plot(x,y,'o',ms=2,mfc=colors[body],mec="white",mew=0.2,alpha=1,zorder=10)
        ax.plot(xOrb[body],yOrb[body],'-',lw=0.5,color=colors[body],zorder=9)
    
    # plot the Asteroids

    ax.plot(xTr,yTr,'o',ms=0.5,mfc='green',mec='green',mew=0.1,alpha=1,zorder=5,label='Jupiter Trojans')
    ax.plot(xH,yH,'o',ms=0.5,mfc='#bb0000',mec='#bb0000',mew=0.1,alpha=1,zorder=6,label='Hildas (3:2 resonance)')

    # and the sun

    ax.plot(0,0,'o',ms=4,mfc='yellow',mec='yellow',mew=0.3,alpha=1,zorder=10)

    # Key

    leg = ax.legend(loc="lower right",frameon=False,prop={'size':5})
    for handle in leg.legend_handles:
        handle.set_markersize(3)
        handle.set_alpha(1.0)

    # Time label at upper left

    ax.text(-0.95*aMax,+0.95*aMax,f"{frameDate}",fontsize=10,ha='left',va='center',color='yellow')

    # save hardcopy

    plt.savefig(frameFile,bbox_inches='tight',facecolor='black')

    plt.close()
    
print(f"Done - created {len(frames)} frames")